In [ ]:
#
# Find all models, sort them (by date), and deploy the latest one
#

models = [Path(model).name for model in s3filesystem.ls(s3_output_path)
        if (Path(model).name.startswith("giia"))]
sorted_models = sorted(models)

endpoint_name = sagemaker_session.endpoint_from_job(
    job_name=sorted_models[0],
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    deployment_image=image_name,
    role=role
)

predictor = chainer_estimator.deploy(initial_instance_count=1, instance_type=instance_type)
# https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/chainer_mnist/chainer_mnist_local_mode.ipynb
